In [1]:
import warnings

warnings.filterwarnings('ignore')

In [2]:
from constants import dataset_metadata, verbose
from task2_apriori import apriori_for_dataset

In [3]:
dataset_name = 'movielens'

In [4]:
dataset_metadata[dataset_name]

{'path': ['./data/movielens/ratings.csv', './data/movielens/movies.csv'],
 'size': 100836,
 'limit': 50000,
 'minsup': [50, 100, 150, 200, 250],
 'threshold_rating': 4.0}

In [5]:
apriori_for_dataset(dataset_name, dataset_metadata, verbose=verbose)

Running Apriori on movielens : minsup = 50
################################################################################
Accessing data-stream #1
Num. of transactions: 322
Num. of freq itemsets in this chunk: 10
Total num. of freq itemsets till now: 10
Finished data-stream #1 in 7.4 seconds.
--------------------------------------------------------------------------------
Accessing data-stream #2
Num. of transactions: 288
Num. of freq itemsets in this chunk: 1
Total num. of freq itemsets till now: 11
Finished data-stream #2 in 6.3 seconds.
--------------------------------------------------------------------------------
Accessing data-stream #3
Num. of transactions: 1
Num. of freq itemsets: 0
Total num. of freq itemsets till now: 11
Finished data-stream #3 in 0.0 seconds.
--------------------------------------------------------------------------------
Time taken for minsup 50 = 0.0 seconds.
Itemsets saved to ./logs/movielens/itemsets_df_minsup_50.csv
----------------------------------